In [10]:
import os
import json
import numpy as np

In [11]:
fw = open("S_14_01_1_O_scan1/S_14_01_1_O_scan1.json",'r')
objects = []
PTSMtx = []
objlocs = []
objdims = []
objlabel = []
filePath="S_14_01_0_scan1"
while True:
    line = fw.readline()
    if not line :
        break
    if("filename" in line):
        filename = line.split(":")[1].split('"')[1]
        print(filename)
    if ("toPTSMtxInverse" in line):
        while True:
            line = fw.readline()
            if("]" in line):
                break
            temp = line.replace(',', '')
            temp = temp.strip()
            PTSMtx.append(temp)
    if("objects" in line):
        while True:
            line = fw.readline()
            if("]" in line): 
                break
            temp_object = line.split('"')[1]
            for i in range(6):
                temp_object = temp_object.replace('\\\\', '/')
            for i in range(3):
                temp_object = temp_object.replace('//', '/')
            temp_object.replace(",",'')
            objects.append(temp_object)

for obj in objects:
    fo = open(filename+"_scan1"+obj, "r")
    while True:
        line = fo.readline()
        if not line:
            break
        if ("label" in line):
            label = line.split('"')[3]                
        if ("bboxMin" in line):            
            line = fo.readline()
            bmin = line.strip()
            line = fo.readline()            
            bmin = bmin + line.strip()
            line = fo.readline()
            bmin = bmin + line.strip()
        if("bboxMax" in line):
            line = fo.readline()
            bmax = line.strip()
            line = fo.readline()            
            bmax = bmax + line.strip()
            line = fo.readline()
            bmax = bmax + line.strip()
    bmins = bmin.split(",")
    bmaxs = bmax.split(",")
    blocs = []
    bdims = []
    for i in range(3):
        blocs.append(round((float(bmins[i]) + float(bmaxs[i]))/2, 3))
        bdims.append(round(float(bmaxs[i]) - float(bmins[i]), 3))
    objlabel.append(label)
    objlocs.append(blocs)
    objdims.append(bdims)
    print("label : " + label)
    print("bloc : "+ str(blocs))
    print("bdim : " + str(bdims))
    fo.close()
fw.close()

S_14_01_1_O
label : Floor
bloc : [33.982, 13.736, 0.445]
bdim : [13.211, 10.805, 0.01]
label : Wall
bloc : [40.593, 13.734, 1.97]
bdim : [0.01, 10.8, 3.06]
label : Wall
bloc : [27.381, 12.628, 1.97]
bdim : [0.01, 7.42, 3.06]
label : Wall
bloc : [27.381, 18.236, 1.97]
bdim : [0.01, 1.796, 3.06]
label : Wall
bloc : [33.987, 19.133, 1.97]
bdim : [13.221, 0.01, 3.06]
label : Wall
bloc : [34.164, 8.339, 1.97]
bdim : [12.857, 0.01, 3.06]
label : Ceiling
bloc : [33.987, 13.731, 3.495]
bdim : [13.201, 10.795, 0.01]
label : Column
bloc : [27.202, 16.838, 1.97]
bdim : [1.0, 1.0, 3.06]
label : Column
bloc : [27.336, 8.518, 1.97]
bdim : [0.8, 0.8, 3.06]
label : Door
bloc : [35.778, 7.941, 1.509]
bdim : [1.624, 1.045, 2.138]
label : Door
bloc : [30.238, 7.941, 1.509]
bdim : [1.624, 1.045, 2.138]
label : Table
bloc : [34.838, 18.817, 0.798]
bdim : [1.198, 0.576, 0.696]
label : Table
bloc : [28.861, 10.71, 0.776]
bdim : [0.579, 2.003, 0.752]
label : Table
bloc : [28.756, 13.56, 0.776]
bdim : [0.579, 

In [19]:
axis_align_matrix = np.array(PTSMtx, dtype=np.float64).reshape((4,4))
print(axis_align_matrix)

[[ 5.83257705e-01 -8.12287172e-01  0.00000000e+00  2.94670159e+05]
 [ 8.12287172e-01  5.83257705e-01  0.00000000e+00 -4.63163190e+05]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -6.13920008e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [20]:
gt_unaligned = []
for i in range(len(objlabel)):
    unaligns = []
    for j in range(3):
        unaligns.append(objlocs[i][j])
    for j in range(3):
        unaligns.append(objdims[i][j])
    gt_unaligned.append(unaligns)

In [53]:
gt_aligned = []
align_locs = objlocs
align_dims = objdims
for i in range(len(objlabel)):
    aligns = []
    pts = np.ones((np.array(align_locs[i]).shape[0],4))
    pts[:, 0:3] = align_locs[i]
    pts = np.dot(pts, axis_align_matrix)
    align_locs[i] = pts[:1, 0:3].reshape(3).tolist()
    pts = np.ones((np.array(align_dims[i]).shape[0],4))
    pts[:, 0:3] = align_dims[i]
    pts = np.dot(pts, axis_align_matrix)    
    align_dims[i] = pts[:1, 0:3].reshape(3).tolist()
    gt_aligned.append(np.concatenate((align_locs[i], align_dims[i]), axis=0).tolist())
print(gt_aligned)

AttributeError: 'list' object has no attribute 'type'

In [54]:
infos={'point_cloud': {'num_features': 6, 'lidar_idx': filename},
      'pts_path' : filePath+"/"+filename+"_scan1.pts",
       "pts_instance_mask_path" : 'instance_mask/'+ filename+"_scan1.pts",
       "pts_semantic_mask_path" : 'semantic_mask/'+filename+"_scan1.pts",
       'annos':{ 'gt_num' : len(objlabel),
                'name' : objlabel,
       },
       'location': align_locs,
       'dimension': align_dims,
       'gt_boxes_upright_depth' : gt_aligned,
       'unaligned_location':objlocs,
       'unaligned_dimensions':objdims,
       'unaligned_gt_boxes_upright_depth': gt_unaligned,
       'index' : np.arange(len(objlabel)).tolist(),
       'class' : "",
       'axis_align_matrix': axis_align_matrix.tolist()
      }
print(json.dumps(infos))

{"point_cloud": {"num_features": 6, "lidar_idx": "S_14_01_1_O"}, "pts_path": "S_14_01_0_scan1/S_14_01_1_O_scan1.pts", "pts_instance_mask_path": "instance_mask/S_14_01_1_O_scan1.pts", "pts_semantic_mask_path": "semantic_mask/S_14_01_1_O_scan1.pts", "annos": {"gt_num": 150, "name": ["Floor", "Wall", "Wall", "Wall", "Wall", "Wall", "Ceiling", "Column", "Column", "Door", "Door", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Table", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Chair", "Lighting", "Lighting", "Lighting", "Lighting", "Lighting", "Lig